In [14]:
CHARACTER = "ФИБИ"

main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']
labels_to_chars = dict(enumerate(main_characters))
chars_to_labels = {v: k for k, v in labels_to_chars.items()}
labels_to_chars, chars_to_labels

({0: 'ДЖОУИ', 1: 'МОНИКА', 2: 'РЕЙЧЕЛ', 3: 'РОСС', 4: 'ФИБИ', 5: 'ЧЕНДЛЕР'},
 {'ДЖОУИ': 0, 'МОНИКА': 1, 'РЕЙЧЕЛ': 2, 'РОСС': 3, 'ФИБИ': 4, 'ЧЕНДЛЕР': 5})

In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [16]:
#!g1.1
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [17]:
!git clone https://github.com/Alenush/style_transfer_sirius2021summer.git

Cloning into 'style_transfer_sirius2021summer'...
remote: Enumerating objects: 1667, done.
remote: Counting objects: 100% (1667/1667), done.
remote: Compressing objects: 100% (1024/1024), done.
remote: Total 1667 (delta 748), reused 1530 (delta 616), pack-reused 0
Receiving objects: 100% (1667/1667), 43.41 MiB | 9.91 MiB/s, done.
Resolving deltas: 100% (748/748), done.


In [20]:
%%bash

cd style_transfer_sirius2021summer
git checkout master
git pull
cd -

Your branch is up to date with 'origin/master'.
Updating 048b0b1..da67de9
Fast-forward
 data/train_data/en/only_unique_phrases_eng.csv     | 59297 +++++++++++++++++++
 .../{Friends_Parser.ipynb => Scripts_Parser.ipynb} |     0
 utils/[Sergey+Polina]_Phrases_Preprocessing.ipynb  | 13604 +++++
 3 files changed, 72901 insertions(+)
 create mode 100644 data/train_data/en/only_unique_phrases_eng.csv
 rename utils/{Friends_Parser.ipynb => Scripts_Parser.ipynb} (100%)
 create mode 100644 utils/[Sergey+Polina]_Phrases_Preprocessing.ipynb
/content


Already on 'master'
From https://github.com/Alenush/style_transfer_sirius2021summer
   048b0b1..da67de9  master     -> origin/master


In [21]:
#!g1.1
df = pd.read_csv("style_transfer_sirius2021summer/data/train_data/en/only_unique_phrases_eng.csv")
df

,label,sentence,base_sentence
0,3,You gotta be at least bi.,you get ta be at least bi
1,0,You're hurting.,youre hurt
2,2,We are not going anywhere!,we be not go anywhere
3,4,"Hey, you guys can I just like have a second al...",hey you guy can i just like have a second alon...
4,1,Why arent you dressed yet?!,why arent you dress yet
...,...,...,...
59291,3,"Uh, we should really do it again sometime, wha...",uh we should really do it again sometime say
59292,0,I don't know what happens if the sea turtle ca...,i dont know what happens if the sea turtle cat...
59293,4,"Yeah, maybe before we start we should just get...",yeah maybe before we start we should just get ...
59294,4,"Hey, oh, so, um how'd you make out last night?",hey oh so um howd you make out last night


In [23]:
df_person = df.copy()
idx = chars_to_labels[CHARACTER]
df_person["label"] = df_person["label"].progress_apply(lambda x: 1 if x == idx else 0)
df_person

100%|██████████| 59296/59296 [00:00<00:00, 671059.67it/s]


,label,sentence,base_sentence
0,0,You gotta be at least bi.,you get ta be at least bi
1,0,You're hurting.,youre hurt
2,0,We are not going anywhere!,we be not go anywhere
3,1,"Hey, you guys can I just like have a second al...",hey you guy can i just like have a second alon...
4,0,Why arent you dressed yet?!,why arent you dress yet
...,...,...,...
59291,0,"Uh, we should really do it again sometime, wha...",uh we should really do it again sometime say
59292,0,I don't know what happens if the sea turtle ca...,i dont know what happens if the sea turtle cat...
59293,1,"Yeah, maybe before we start we should just get...",yeah maybe before we start we should just get ...
59294,1,"Hey, oh, so, um how'd you make out last night?",hey oh so um howd you make out last night


In [25]:
import re

df_person["not_alpha_num_punct"] = df_person["sentence"].apply(lambda x: bool(re.match(r"[A-Za-z0-9 _.,!\"\'/$]*", x)))
df_person

,label,sentence,base_sentence,not_alpha_num_punct
0,0,You gotta be at least bi.,you get ta be at least bi,True
1,0,You're hurting.,youre hurt,True
2,0,We are not going anywhere!,we be not go anywhere,True
3,1,"Hey, you guys can I just like have a second al...",hey you guy can i just like have a second alon...,True
4,0,Why arent you dressed yet?!,why arent you dress yet,True
...,...,...,...,...
59291,0,"Uh, we should really do it again sometime, wha...",uh we should really do it again sometime say,True
59292,0,I don't know what happens if the sea turtle ca...,i dont know what happens if the sea turtle cat...,True
59293,1,"Yeah, maybe before we start we should just get...",yeah maybe before we start we should just get ...,True
59294,1,"Hey, oh, so, um how'd you make out last night?",hey oh so um howd you make out last night,True


In [26]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.6 MB/s 
     |████████████████████████████████| 895 kB 59.9 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
     |████████████████████████████████| 636 kB 53.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [27]:
df_person[df_person["label"] == 1]

,label,sentence,base_sentence,not_alpha_num_punct
3,1,"Hey, you guys can I just like have a second al...",hey you guy can i just like have a second alon...,True
5,1,Cut her out of our lives!,cut her out of our life,True
6,1,"Well, just, let's try it again.",well just let try it again,True
12,1,"So I looked her up, and she lives out by the b...",so i look her up and she life out by the beach,True
14,1,She really wants to make up!,she really want to make up,True
...,...,...,...,...
59273,1,Almost wore my threadbare robe that can't cont...,almost wore my threadbare robe that cant conta...,True
59275,1,"Ooh, this is so exciting!",ooh this be so excite,True
59278,1,You gotta do something about your eyes.,you get ta do something about your eye,True
59293,1,"Yeah, maybe before we start we should just get...",yeah maybe before we start we should just get ...,True


In [28]:
sentences = df_person.sentence.values
labels = df_person.label.values

In [29]:
from transformers import BertTokenizer
# using the low level BERT for our task.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Printing the original sentence.
print(' Original: ', sentences[0])

# Printing the tokenized sentence in form of list.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))


 Original:  You gotta be at least bi.
Tokenized:  ['you', 'gotta', 'be', 'at', 'least', 'bi', '.']
Token IDs:  [2017, 10657, 2022, 2012, 2560, 12170, 1012]


In [30]:
input_ids = []
for sent in sentences:
    # so basically encode tokenizing , mapping sentences to thier token ids after adding special tokens.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence which are encoding.
                        add_special_tokens = True, # Adding special tokens '[CLS]' and '[SEP]'

                         )
    
 
    input_ids.append(encoded_sent)

In [31]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 128

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN , truncating="post", padding="post")

In [32]:
attention_masks = []

for sent in input_ids:
    
    # Generating attention mask for sentences.
    #   - when there is 0 present as token id we are going to set mask as 0.
    #   - we are going to set mask 1 for all non-zero positive input id.
    att_mask = [int(token_id > 0) for token_id in sent]
    
   
    attention_masks.append(att_mask)

In [33]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,test_size=0.2)

In [34]:
#changing the numpy arrays into tensors for working on GPU. 
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Deciding the batch size for training.

batch_size = 32

#DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# DataLoader for our validation(test) set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [35]:
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,   
    output_attentions = False,
    output_hidden_states = False,
)

# Running the model on GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [36]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

In [37]:
#!g1.1
from transformers import get_linear_schedule_with_warmup
import time

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [41]:
from tqdm import trange

In [42]:
loss_values = []

for epoch_i in trange(int(epochs), desc="Epoch"):
    t0 = time.time()

    total_loss = 0
    # putting model in traing mode there are two model eval and train for model
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        
        #checking time taken after every 50 steps.
        if step % 50 == 0 and not step == 0:
            elapsed = (time.time() - t0)
        
        #getting ids,mask,labes for every batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

        loss = outputs[0]

        total_loss += loss.item()
        
        # doing back propagation
        loss.backward()

        print("\r%f" % loss, end='')

        optimizer.step()
        
        # Update the learning rate.
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader) 
    loss_values.append(avg_train_loss)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

0.346960

Epoch:  25%|██▌       | 1/4 [16:30<49:30, 990.28s/it]

0.673638

Epoch:  50%|█████     | 2/4 [33:03<33:02, 991.17s/it]

1.085719

Epoch:  75%|███████▌  | 3/4 [49:36<16:31, 991.79s/it]

0.243340

Epoch: 100%|██████████| 4/4 [1:06:09<00:00, 992.46s/it]

1.067336

In [ ]:
chars_ru2en = {
    'ДЖОУИ'  : 'Joey',
    'МОНИКА' : 'Monica',
    'РЕЙЧЕЛ' : 'Rachel',
    'РОСС'   : 'Ross',
    'ФИБИ'   : 'Phoebe',
    'ЧЕНДЛЕР': 'Chandler',
}

In [ ]:
chars_ru2en

In [ ]:
import pathlib

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = f'en-{chars_ru2en[CHARACTER].lower()}'

ROOT_DIR = '/content/drive/MyDrive/FriendsClassifiers'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'{ROOT_DIR}/outputs/{TASK_NAME}/'
pathlib.Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
import os
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

In [ ]:
CONFIG_NAME = "config.json"
WEIGHTS_NAME = f"{TASK_NAME}-pytorch_model.bin"